In [30]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics.pairwise import haversine_distances
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_excel("data.xlsx")

In [5]:
df_copy = df.copy()

In [76]:
df = df_copy.copy()
df.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668640009,housing/rent/apartment,One BR 507 & 509 Esplanade,"This unit is located at 507 & 509 Esplanade, R...",NaN,1,1,USD,No,Thumbnail,...,2195,Monthly,542,507 509 Esplanade,Redondo Beach,CA,33.8520,-118.3759,RentLingo,1.577360e+09
1,5668639818,housing/rent/apartment,Three BR 146 Lochview Drive,"This unit is located at 146 Lochview Drive, Ne...",NaN,1.5,3,USD,No,Thumbnail,...,1250,Monthly,1500,146 Lochview Dr,Newport News,VA,37.0867,-76.4941,RentLingo,1.577360e+09
2,5668639686,housing/rent/apartment,Three BR 3101 Morningside Drive,This unit is located at 3101 Morningside Drive...,NaN,2,3,USD,No,Thumbnail,...,1395,Monthly,1650,3101 Morningside Dr,Raleigh,NC,35.8230,-78.6438,RentLingo,1.577360e+09
3,5668639659,housing/rent/apartment,Two BR 209 Aegean Way,"This unit is located at 209 Aegean Way, Vacavi...",NaN,1,2,USD,No,Thumbnail,...,1600,Monthly,820,209 Aegean Way,Vacaville,CA,38.3622,-121.9712,RentLingo,1.577360e+09
4,5668639374,housing/rent/apartment,One BR 4805 Marquette NE,"This unit is located at 4805 Marquette NE, Alb...",NaN,1,1,USD,No,Thumbnail,...,975,Monthly,624,4805 Marquette NE,Albuquerque,NM,35.1038,-106.6110,RentLingo,1.577360e+09


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99826 entries, 0 to 99825
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             99826 non-null  object 
 1   category       99826 non-null  object 
 2   title          99826 non-null  object 
 3   body           99812 non-null  object 
 4   amenities      83749 non-null  object 
 5   bathrooms      99760 non-null  object 
 6   bedrooms       99699 non-null  object 
 7   currency       99822 non-null  object 
 8   fee            99823 non-null  object 
 9   has_photo      99823 non-null  object 
 10  pets_allowed   39192 non-null  object 
 11  price          99821 non-null  float64
 12  price_display  99820 non-null  object 
 13  price_type     99823 non-null  object 
 14  square_feet    99823 non-null  object 
 15  address        7946 non-null   object 
 16  cityname       99521 non-null  object 
 17  state          99521 non-null  object 
 18  latitu

In [78]:
df = df.drop(columns=["id","category","title","body","time","address","currency","price_display"])

In [79]:
df.columns

Index(['amenities', 'bathrooms', 'bedrooms', 'fee', 'has_photo',
       'pets_allowed', 'price', 'price_type', 'square_feet', 'cityname',
       'state', 'latitude', 'longitude', 'source'],
      dtype='object')

In [80]:
df['amenities'] =  df['amenities'].fillna("No amenities")
test = set()
for value in df['amenities'].unique():
    temp = value.split(",")
    for var in temp:
        test.add(var)

for column in test:
    df[column] = df['amenities'].apply(lambda x: 1 if column in x else 0)

df = df.drop(columns=['amenities','No','USD'])


In [81]:
df['pets_allowed'] = df['pets_allowed'].fillna("No")
df['pets_allowed'].isna().sum()

0

In [82]:
df = df.dropna()

In [83]:
df = df[df['bathrooms']!="Thumbnail"]
df = df[df['bedrooms']!="Thumbnail"]
df = df[df['fee'].isin(["Yes", "No"])]
df['has_photo'] = df['has_photo'].replace({"Thumbnail":"No"})
df['pets_allowed'] = df['pets_allowed'].apply(lambda x: 1 if "Cats" in x or "Dogs" in x else 0)

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99335 entries, 0 to 99825
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bathrooms           99335 non-null  object 
 1   bedrooms            99335 non-null  object 
 2   fee                 99335 non-null  object 
 3   has_photo           99335 non-null  object 
 4   pets_allowed        99335 non-null  int64  
 5   price               99335 non-null  float64
 6   price_type          99335 non-null  object 
 7   square_feet         99335 non-null  object 
 8   cityname            99335 non-null  object 
 9   state               99335 non-null  object 
 10  latitude            99335 non-null  float64
 11  longitude           99335 non-null  float64
 12  source              99335 non-null  object 
 13  Garbage Disposal    99335 non-null  int64  
 14  Gated               99335 non-null  int64  
 15  Clubhouse           99335 non-null  int64  
 16  Parking  

In [85]:
for column in df.columns: 
    if column in ['bathrooms','bedrooms', 'price','latitude','longitude', 'square_feet']:
        df[column] = df[column].astype('float')
    else:
        df[column] = df[column].astype('object')

In [86]:
Regression_comparision = {}

In [87]:
#Benchmark models
X1 = df.drop(columns=['price','cityname','state']).copy()
y1 = df['price']
X1 = pd.get_dummies(X1, columns=['fee','has_photo','pets_allowed','price_type','source'], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.4, random_state=1234)

In [88]:
#Linear regression
lr_benchmark = LinearRegression()
lr_benchmark.fit(X_train, y_train)

y_pred = lr_benchmark.predict(X_test)

r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)
MAPE = mean_absolute_percentage_error(y_test, y_pred)

Regression_comparision['LinearRegression_benchmark'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [89]:
#Ridge
ridge_benchmark = Ridge()
ridge_benchmark.fit(X_train, y_train)

y_pred = ridge_benchmark.predict(X_test)

r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)
MAPE = mean_absolute_percentage_error(y_test, y_pred)

Regression_comparision['Ridge_benchmark'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [90]:
#Lasso
lasso_benchmark = Lasso()
lasso_benchmark.fit(X_train, y_train)

y_pred = lasso_benchmark.predict(X_test)

r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)
MAPE = mean_absolute_percentage_error(y_test, y_pred)

Regression_comparision['Lasso_benchmark'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [91]:
#Decision Tree benchmark
dc_benchmark = DecisionTreeRegressor()
dc_benchmark.fit(X_train, y_train)

y_pred = dc_benchmark.predict(X_test)

r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)
MAPE = mean_absolute_percentage_error(y_test, y_pred)

Regression_comparision['DecisionTree_benchmark'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [92]:
#Random Forest benchmark
rf_benchmark = RandomForestRegressor()
rf_benchmark.fit(X_train, y_train)

y_pred = rf_benchmark.predict(X_test)

r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)
MAPE = mean_absolute_percentage_error(y_test, y_pred)

Regression_comparision['RandomForest_benchmark'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [93]:
#MLP benchmark
mlp_benchmark = MLPRegressor(max_iter=500)
mlp_benchmark.fit(X_train, y_train)

y_pred = mlp_benchmark.predict(X_test)

r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)
MAPE = mean_absolute_percentage_error(y_test, y_pred)

Regression_comparision['MultilayerPerceptron_benchmark'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [104]:
# Second ilteration
X2 = X1.copy()
y2 = y1.copy()

#  Using clustering to cluster the lat/long coordinates into meaningful clusters
data = df[['latitude','longitude']].copy()

# Convert latitude and longitude from degrees to radians
data['latitude_radians'] = np.radians(data['latitude'])
data['longitude_radians'] = np.radians(data['longitude'])

# Create a 2D array of coordinates in radians
coords = data[['latitude_radians', 'longitude_radians']].values

# Apply DBSCAN using haversine distance
dbscan = DBSCAN(eps=0.005, min_samples=10, metric='haversine')
X2['cluster'] = dbscan.fit_predict(coords)

X2 = X2.drop(columns=['longitude','latitude'])

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2, test_size = 0.4, random_state=1234)

In [96]:
# Linear regression
lr_clustering = LinearRegression()
lr_clustering.fit(X_train2, y_train2)

y_pred = lr_clustering.predict(X_test2)

r2 = r2_score(y_test2, y_pred)
MSE = mean_squared_error(y_test2, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test2, y_pred)
MAPE = mean_absolute_percentage_error(y_test2, y_pred)

Regression_comparision['LinearRegression_clustering'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [97]:
# Ridge
ridge_clustering = Ridge()
ridge_clustering.fit(X_train2, y_train2)

y_pred = ridge_clustering.predict(X_test2)

r2 = r2_score(y_test2, y_pred)
MSE = mean_squared_error(y_test2, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test2, y_pred)
MAPE = mean_absolute_percentage_error(y_test2, y_pred)

Regression_comparision['Ridge_clustering'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [98]:
# Lasso
lasso_clustering = Lasso()
lasso_clustering.fit(X_train2, y_train2)

y_pred = lasso_clustering.predict(X_test2)

r2 = r2_score(y_test2, y_pred)
MSE = mean_squared_error(y_test2, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test2, y_pred)
MAPE = mean_absolute_percentage_error(y_test2, y_pred)

Regression_comparision['Lasso_clustering'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [99]:
#Decision Tree 
dc_clustering = DecisionTreeRegressor()
dc_clustering.fit(X_train2, y_train2)

y_pred = dc_clustering.predict(X_test2)

r2 = r2_score(y_test2, y_pred)
MSE = mean_squared_error(y_test2, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test2, y_pred)
MAPE = mean_absolute_percentage_error(y_test2, y_pred)

Regression_comparision['DecisionTree_clustering'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [100]:
#Random Forest 
rf_clustering = RandomForestRegressor()
rf_clustering.fit(X_train2, y_train2)

y_pred = rf_clustering.predict(X_test2)

r2 = r2_score(y_test2, y_pred)
MSE = mean_squared_error(y_test2, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test2, y_pred)
MAPE = mean_absolute_percentage_error(y_test2, y_pred)

Regression_comparision['RandomForest_clustering'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [101]:
#MLP
mlp_clustering = MLPRegressor(max_iter=500)
mlp_clustering.fit(X_train2, y_train2)

y_pred = mlp_clustering.predict(X_test2)

r2 = r2_score(y_test2, y_pred)
MSE = mean_squared_error(y_test2, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test2, y_pred)
MAPE = mean_absolute_percentage_error(y_test2, y_pred)

Regression_comparision['MultilayerPerceptron_clustering'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [105]:
# Third ilteration
X3 = X1.copy()
y3 = y1.copy()

#  Using clustering to cluster the lat/long coordinates into meaningful clusters
data = df[['latitude','longitude']].copy()

# Apply DBSCAN 
dbscan = DBSCAN(eps=0.2, min_samples=10)
X3['cluster'] = dbscan.fit_predict(X3[['latitude', 'longitude']])

X3 = X3.drop(columns=['longitude','latitude'])

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3,y3, test_size = 0.4, random_state=1234)

In [106]:
X3['cluster'].value_counts()

cluster
8      8721
0      8272
46     7243
7      5732
5      5445
       ... 
116      10
200      10
231      10
201      10
204      10
Name: count, Length: 236, dtype: int64

In [107]:
# Linear regression
lr_clustering_euclidean = LinearRegression()
lr_clustering_euclidean.fit(X_train3, y_train3)

y_pred = lr_clustering_euclidean.predict(X_test3)

r2 = r2_score(y_test3, y_pred)
MSE = mean_squared_error(y_test3, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test3, y_pred)
MAPE = mean_absolute_percentage_error(y_test3, y_pred)

Regression_comparision['LinearRegression_clustering_euclidean'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [109]:
# Ridge regression
ridge_clustering_euclidean = Ridge()
ridge_clustering_euclidean.fit(X_train3, y_train3)

y_pred = ridge_clustering_euclidean.predict(X_test3)

r2 = r2_score(y_test3, y_pred)
MSE = mean_squared_error(y_test3, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test3, y_pred)
MAPE = mean_absolute_percentage_error(y_test3, y_pred)

Regression_comparision['Ridge_clustering_euclidean'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [114]:
# Lasso regression
lasso_clustering_euclidean = Lasso()
lasso_clustering_euclidean.fit(X_train3, y_train3)

y_pred = lasso_clustering_euclidean.predict(X_test3)

r2 = r2_score(y_test3, y_pred)
MSE = mean_squared_error(y_test3, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test3, y_pred)
MAPE = mean_absolute_percentage_error(y_test3, y_pred)

Regression_comparision['Lasso_clustering_euclidean'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [116]:
# Decision tree regression
dc_clustering_euclidean = DecisionTreeRegressor()
dc_clustering_euclidean.fit(X_train3, y_train3)

y_pred = dc_clustering_euclidean.predict(X_test3)

r2 = r2_score(y_test3, y_pred)
MSE = mean_squared_error(y_test3, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test3, y_pred)
MAPE = mean_absolute_percentage_error(y_test3, y_pred)

Regression_comparision['DecisionTree_clustering_euclidean'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [118]:
# Random forest regression
rf_clustering_euclidean = RandomForestRegressor()
rf_clustering_euclidean.fit(X_train3, y_train3)

y_pred = rf_clustering_euclidean.predict(X_test3)

r2 = r2_score(y_test3, y_pred)
MSE = mean_squared_error(y_test3, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test3, y_pred)
MAPE = mean_absolute_percentage_error(y_test3, y_pred)

Regression_comparision['RandomForest_clustering_euclidean'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [120]:
# MLP regression
mlp_clustering_euclidean = MLPRegressor()
mlp_clustering_euclidean.fit(X_train3, y_train3)

y_pred = mlp_clustering_euclidean.predict(X_test3)

r2 = r2_score(y_test3, y_pred)
MSE = mean_squared_error(y_test3, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test3, y_pred)
MAPE = mean_absolute_percentage_error(y_test3, y_pred)

Regression_comparision['MultilayerPerceptron_clustering_euclidean'] = {'r2': r2,
                                                        'MSE': MSE,
                                                        'RMSE': RMSE,
                                                        'MAE': MAE,
                                                        'MAPE': MAPE}

In [122]:
results = pd.DataFrame(Regression_comparision)
results

,LinearRegression_benchmark,Ridge_benchmark,Lasso_benchmark,DecisionTree_benchmark,RandomForest_benchmark,MultilayerPerceptron_benchmark,LinearRegression_clustering,Ridge_clustering,Lasso_clustering,DecisionTree_clustering,RandomForest_clustering,MultilayerPerceptron_clustering,LinearRegression_clustering_euclidean,Ridge_clustering_euclidean,Lasso_clustering_euclidean,DecisionTree_clustering_euclidean,RandomForest_clustering_euclidean,MultilayerPerceptron_clustering_euclidean
r2,0.276934,0.277420,0.272144,0.634769,0.775482,0.634769,0.320070,0.320621,0.315914,0.378042,0.663129,0.353126,0.309342,0.309789,0.304916,0.328865,0.673813,0.341421
MSE,529529.728212,529174.156766,533037.657409,267472.904220,164423.282878,267472.904220,497939.840024,497536.364796,500983.419425,455484.058289,246704.216207,473731.358975,505796.014059,505468.714473,509037.745133,491498.761319,238879.921925,482303.176056
RMSE,727.687933,727.443576,730.094280,517.177826,405.491409,517.177826,705.648524,705.362577,707.801822,674.895591,496.693282,688.281453,711.193373,710.963230,713.468812,701.069726,488.753437,694.480508
MAE,476.290444,476.162735,477.670214,222.759992,173.367587,222.759992,453.511727,453.457559,454.578687,310.996897,248.433204,427.663503,453.537389,453.454367,454.440761,302.716933,240.632644,422.368063
MAPE,0.339997,0.339903,0.341532,0.144706,0.114266,0.144706,0.320819,0.320814,0.321925,0.195453,0.159832,0.295321,0.319143,0.319110,0.320065,0.189354,0.154833,0.284259


In [123]:
results.filter(like='LinearRegression')

,LinearRegression_benchmark,LinearRegression_clustering,LinearRegression_clustering_euclidean
r2,0.276934,0.320070,0.309342
MSE,529529.728212,497939.840024,505796.014059
RMSE,727.687933,705.648524,711.193373
MAE,476.290444,453.511727,453.537389
MAPE,0.339997,0.320819,0.319143


In [124]:
results.filter(like='Lasso')

,Lasso_benchmark,Lasso_clustering,Lasso_clustering_euclidean
r2,0.272144,0.315914,0.304916
MSE,533037.657409,500983.419425,509037.745133
RMSE,730.094280,707.801822,713.468812
MAE,477.670214,454.578687,454.440761
MAPE,0.341532,0.321925,0.320065


In [125]:
results.filter(like='Ridge')

,Ridge_benchmark,Ridge_clustering,Ridge_clustering_euclidean
r2,0.277420,0.320621,0.309789
MSE,529174.156766,497536.364796,505468.714473
RMSE,727.443576,705.362577,710.963230
MAE,476.162735,453.457559,453.454367
MAPE,0.339903,0.320814,0.319110


In [126]:
results.filter(like='DecisionTree')

,DecisionTree_benchmark,DecisionTree_clustering,DecisionTree_clustering_euclidean
r2,0.634769,0.378042,0.328865
MSE,267472.904220,455484.058289,491498.761319
RMSE,517.177826,674.895591,701.069726
MAE,222.759992,310.996897,302.716933
MAPE,0.144706,0.195453,0.189354


In [127]:
results.filter(like='RandomForest')

,RandomForest_benchmark,RandomForest_clustering,RandomForest_clustering_euclidean
r2,0.775482,0.663129,0.673813
MSE,164423.282878,246704.216207,238879.921925
RMSE,405.491409,496.693282,488.753437
MAE,173.367587,248.433204,240.632644
MAPE,0.114266,0.159832,0.154833


In [129]:
results.filter(like='MultilayerPerceptron')

,MultilayerPerceptron_benchmark,MultilayerPerceptron_clustering,MultilayerPerceptron_clustering_euclidean
r2,0.634769,0.353126,0.341421
MSE,267472.904220,473731.358975,482303.176056
RMSE,517.177826,688.281453,694.480508
MAE,222.759992,427.663503,422.368063
MAPE,0.144706,0.295321,0.284259
